# 04 QCPG Paraphrase
- Input: `03_augment_llm.csv`
- Output: `04_qcpg_paraphrase.csv`

In [1]:
import gc
import platform
import pandas as pd

import torch

from datasets import Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

from tqdm.notebook import tqdm

In [2]:
# If on mac default to mps, otherwise cuda, then cpu
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    # Check if on mac
    if platform.system() == "Darwin":
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
device

device(type='mps')

## Load data

In [3]:
df = pd.read_csv("data/03_augment_llm.csv")
df

,question,answers,source,id,en_aug,th_aug,context,en_llm_gec_aug,en_llm_paraphrase_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p..."
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ..."
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,Who is the father of Cleopatra?,Who is Cleopatra's biological father?
...,...,...,...,...,...,...,...,...,...
16975,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,Cartoon book How many fire warriors?,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,How many Fire Warriors are in the cartoon book?,"In the cartoon book, how many fire warriors ar..."
16976,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,When was Bioshock Infin first launched?,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,When was Bioshock Infinite first launched?,When was the first launch of Bioshock Infinite?
16977,ยู ซึง-โฮ เริ่มเข้าวงการบันเทิงเมื่อไหร่?,"{'text': ['ปี 1999', '1999'], 'answer_start': ...",tydiqa,d131cf44b5a08630e822cbf4c33576a434095f6ff4184f...,When did you start the entertainment industry?,คุณเริ่มอุตสาหกรรมบันเทิงเมื่อใด?,ยู ซึง-โฮเดบิวต์เมื่อปี 1999 กับผลงานโฆษณาของ ...,When did you start in the entertainment industry?,When did you begin your career in the entertai...
16978,นภัทร อินทร์ใจเอื้อมีมารดาชื่อว่าอะไร?,"{'text': ['นางวรรณา อินทร์ใจเอื้อ', 'นางวรรณา ...",tydiqa,5e2f4e69eb82918a55969ec99b1c93ae469c264c81c77a...,Naphat Injai Ua has a mother named?,Naphat Injai UA มีแม่ชื่อหรือไม่?,กันเกิดเมื่อวันที่ 23 ตุลาคม พ.ศ. 2533 จังหวัด...,What is the name of Naphat Injai Ua's mother?,What is the name of Naphat Injai Ua's mother?


In [4]:
pipe = pipeline('text2text-generation', model=f'ibm/qcpg-questions', device=device)

In [5]:
def run_augmentation(df, column_name, sem, lex, syn):
    df = df.copy(deep=True)
    df[column_name] = f"COND_SEMANTIC_SIM_{int(sem*100)} COND_LEXICAL_DIV_{int(lex*100)} COND_SYNTACTIC_DIV_{int(syn*100)} " + df[column_name]
    qcpg_augmented = []

    dataset = Dataset.from_pandas(df)

    for out in tqdm(pipe(KeyDataset(dataset, column_name), batch_size=64), desc=f"Running QCPG", total=len(df)):
        qcpg_augmented.append(out[0]["generated_text"])
    
    gc.collect()
    
    return qcpg_augmented

In [6]:
settings_values = [0.2, 0.5, 0.8] 
columns = ["en_aug", "en_llm_gec_aug"]
for column in tqdm(columns, desc="Augmenting Columns"):
    for value in tqdm(settings_values, desc="Augmenting Column Values"):
        if column == "en_aug":
            target_name = f"en_qcpg_{value}_aug"
        else:
            target_name = f"en_qcpg_{value}_{'_'.join(column.split('_')[1:-1])}_aug"
        
        df[target_name] = run_augmentation(df, column, value, value, value)

Augmenting Columns:   0%|          | 0/2 [00:00<?, ?it/s]

Augmenting Column Values:   0%|          | 0/3 [00:00<?, ?it/s]

Running QCPG:   0%|          | 0/16980 [00:00<?, ?it/s]

/Users/parinzee/mambaforge/envs/qgresearch39/lib/python3.9/site-packages/transformers/generation/utils.py:2389: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343673238/work/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:


Running QCPG:   0%|          | 0/16980 [00:00<?, ?it/s]

Running QCPG:   0%|          | 0/16980 [00:00<?, ?it/s]

Augmenting Column Values:   0%|          | 0/3 [00:00<?, ?it/s]

Running QCPG:   0%|          | 0/16980 [00:00<?, ?it/s]

Running QCPG:   0%|          | 0/16980 [00:00<?, ?it/s]

Running QCPG:   0%|          | 0/16980 [00:00<?, ?it/s]

In [7]:
# Sanity check for no empty rows
assert df.isnull().sum().sum() == 0
df

,question,answers,source,id,en_aug,th_aug,context,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,en_qcpg_0.5_aug,en_qcpg_0.8_aug,en_qcpg_0.2_llm_gec_aug,en_qcpg_0.5_llm_gec_aug,en_qcpg_0.8_llm_gec_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,What is the birth date of phuket khan?,What is the date of birth of phattin sarutin?,What is the birth date of phuket khan?,What is phatthira sarutpah ng date of,What is the date of birth of phatthira sarutin?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,What are you playing when you are sarutina?,What are you playing with phatthira sarutong?,What are you playing when you are a sarutin?,What are you playing when you are a phatthira ...,What are you playing in phatthira sarutong ph
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,What career path did phatthira teeratha phikin,What is the career of phatthira teeratha phi,What is the career of a physicist?,What career path did phatthira teeratha phikin,What is the career of phatthira teeratha phi
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,What country did phatthira sarutpah ph,Phatthira sarutpong phokin graduated from?,What year did phatthira sarutpah ph,What year did phatthira sarutpah ph,What country did phatthira sarutpong phokin
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,Who is the father of Cleopatra?,Who is Cleopatra's biological father?,What was cleopatras fathers name?,Who was cleopatras father and mother?,Who was cleopatras father?,What is cleopatras fathers name in greek mytho...,Why was cleopatra called cleopatra?,Who was cleopatras father?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,Cartoon book How many fire warriors?,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,How many Fire Warriors are in the cartoon book?,"In the cartoon book, how many fire warriors ar...",How many warriors in the fire warrior?,How many warriors are in the fire warriors book?,How many fire warriors in the book 'fire warri...,How many fire warrior cartoons are there?,How many f

In [8]:
# Export and save QCPG
df.to_csv("data/04_qcpg_paraphrase.csv", index=False)